In [1]:
from awpy import Demo

import torch
from torch_geometric_temporal.signal import DynamicHeteroGraphTemporalSignal

import pandas as pd
import polars as pl
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

from joblib import load, dump
import random
import json
import sys
import os

pd.set_option('display.max_columns', 100)
pl.Config.set_tbl_rows(10)

sys.path.append(os.path.abspath('../../package'))

from CS2.graph import TabularGraphSnapshot, HeteroGraphSnapshot, TemporalHeteroGraphSnapshot
from CS2.token import Tokenizer
from CS2.preprocess import Dictionary, NormalizePosition, NormalizeTabularGraphSnapshot, ImputeTabularGraphSnapshot
from CS2.visualize import HeteroGraphVisualizer

### 1. Tabular dataset creation

#### 2024 Matches

In [24]:
MATCH_FOLDER_PATH = '../../data/matches-raw/cs2/2024/'
SAVE_PATH = '../../data/matches_processed/cs2/tabular/2024/'

# Get inferno matches
match_list = os.listdir(MATCH_FOLDER_PATH)
inferno_match_list = [file for file in match_list if 'inferno' in file and 
    '-p1' not in file and 
    '-p2' not in file and 
    '-p3' not in file and 
    '-p4' not in file and 
    '-p5' not in file
]

# Save complete match list
parsed_matches_list = []

# Match index
match_index = 0

# Parse Inferno matches
for match in inferno_match_list:

    # Match path
    match_path = MATCH_FOLDER_PATH + match

    # Map nodes dataset
    nodes = pd.read_csv('../../data/map_graph_model/de_inferno/nodes.csv')
    edges = pd.read_csv('../../data/map_graph_model/de_inferno/edges.csv')

    # Create tabular snapshot object
    tg = TabularGraphSnapshot()

    # Create tabular snapshot database
    df, df_dict, active_infernos, active_smokes, active_he_smokes = tg.process_match(
        match_path=match_path,
        player_stats_data_path='../../data/player-stats/scraped-in-2024/2023/norm_player_stats_2023.csv',
        missing_player_stats_data_path='../../data/player-stats/missing_players_df_2024.csv',
        weapon_data_path='../../data/weapon_info/ammo_info.csv',

        ticks_per_second=4,
        numerical_match_id=100000 + match_index,
        num_permutations_per_round=1,
        build_dictionary=True,

        package='polars'
    )

    # Impute missing values
    its = ImputeTabularGraphSnapshot()
    df = its.impute(df)

    # Tokenize match
    tokenizer = Tokenizer()
    df = tokenizer.tokenize_match(df.to_pandas(), 'de_inferno', nodes)

    # Save dataframes
    df.to_csv(SAVE_PATH + match + '_df.csv', index=False)
    df_dict.to_csv(SAVE_PATH + match + '_df_dict.csv', index=False)
    active_infernos.to_csv(SAVE_PATH + match + '_active_infernos.csv', index=False)
    active_smokes.to_csv(SAVE_PATH + match + '_active_smokes.csv', index=False)
    active_he_smokes.to_csv(SAVE_PATH + match + '_active_he_smokes.csv', index=False)


    # Increase index
    match_index += 1

    print(match + ' completed')

../../data/matches-raw/cs2/2024/_betboom-dacha-belgrade-season-2-eternal-fire-vs-pain-bo3-EsO1ghXhDey0Ii85scyL6seternal-fire-vs-pain-m1-inferno.dem
../../data/matches-raw/cs2/2024/_betboom-dacha-belgrade-season-2-mouz-vs-eternal-fire-bo3-5Hd0ELRRmki7NgbHcMS0LRmouz-vs-eternal-fire-m1-inferno.dem
../../data/matches-raw/cs2/2024/_betboom-dacha-belgrade-season-2-virtuspro-vs-furia-bo3-L9mXEKkAd1k6NSuXJ6zD7Mvirtus-pro-vs-furia-m1-inferno.dem
../../data/matches-raw/cs2/2024/_blast-premier-fall-groups-2024-astralis-vs-ninjas-in-pyjamas-bo3-K9s9BWKc9ebp95rNa21UfDastralis-vs-ninjas-in-pyjamas-m2-inferno.dem
../../data/matches-raw/cs2/2024/_blast-premier-fall-groups-2024-faze-vs-cloud9-bo3-ihmdAcNeCnnXz-kUWDRQWJfaze-vs-cloud9-m2-inferno.dem
../../data/matches-raw/cs2/2024/_blast-premier-fall-groups-2024-faze-vs-ninjas-in-pyjamas-bo3-VdTvo-LqKO90fOrCjapEhefaze-vs-ninjas-in-pyjamas-m2-inferno.dem
../../data/matches-raw/cs2/2024/_blast-premier-fall-groups-2024-g2-vs-ninjas-in-pyjamas-bo3-bsSNzyWjPg